In [1]:
import torch
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch.optim import Adam
import torch.nn as nn

/Users/muhammadwaseem/miniconda3/envs/torch/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/muhammadwaseem/miniconda3/envs/torch/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <CAE66874-17C2-35C9-9C4D-6BA9770AF17F> /Users/muhammadwaseem/miniconda3/envs/torch/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <459875AA-DE2C-366B-9C44-90D4B3887080> /Users/muhammadwaseem/miniconda3/envs/torch/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


## Data loading

In [2]:
path = './'

mnist_train_data = MNIST(root = path,
                         train = True,
                         transform = ToTensor(),
                         download = True)
mnist_test_data = MNIST(root = path,
                        train = False,
                        transform = ToTensor(),
                        download = True)

In [3]:
mnist_train_data, mnist_test_data

(Dataset MNIST
     Number of datapoints: 60000
     Root location: ./
     Split: Train
     StandardTransform
 Transform: ToTensor(),
 Dataset MNIST
     Number of datapoints: 10000
     Root location: ./
     Split: Test
     StandardTransform
 Transform: ToTensor())

In [4]:
batch_size = 64
data = DataLoader(mnist_train_data, batch_size=batch_size, shuffle=True)

## Model

In [5]:
# mnist_train_data[0] --> tuple containing input and label of 1st data
# mnist_train_data[0][0] --> input image
input_img = mnist_train_data[0][0].shape
input_img

torch.Size([1, 28, 28])

In [6]:
output_label = mnist_train_data[0][1]
output_label

5

In [7]:
hidden_units = [32, 16]
input_s = input_img[1] * input_img[2]
output_s = 10

layers = [nn.Flatten()] # nn.Flatten(1) -> default param 1
for hidden in hidden_units:
    layers.append(nn.Linear(input_s, hidden))
    layers.append(nn.ReLU())
    input_s = hidden
layers.append(nn.Linear(hidden_units[-1], output_s))
#layers.append(nn.Softmax(dim=1))
layers

[Flatten(start_dim=1, end_dim=-1),
 Linear(in_features=784, out_features=32, bias=True),
 ReLU(),
 Linear(in_features=32, out_features=16, bias=True),
 ReLU(),
 Linear(in_features=16, out_features=10, bias=True)]

In [8]:
torch.manual_seed(1)
Model = nn.Sequential(*layers)
Model

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=32, bias=True)
  (2): ReLU()
  (3): Linear(in_features=32, out_features=16, bias=True)
  (4): ReLU()
  (5): Linear(in_features=16, out_features=10, bias=True)
)

In [9]:
loss_func = nn.CrossEntropyLoss()
optimizer = Adam(Model.parameters(), lr=0.003)

In [10]:
epoch = 20
Training_loss = [0] * epoch
Training_accuracy = [0] * epoch
Test_loss = [0] * epoch
Test_accuracy = [0] * epoch
for i in range(epoch):
    for x_batch, y_batch in data:
        #print(x_batch.shape, y_batch.shape)
        pred = Model(x_batch)
        #print(pred.shape, y_batch.shape)
        loss = loss_func(pred, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        Training_loss[i] += loss.item() * x_batch.size(0)
        softmax = torch.softmax(pred, axis=1)
        correct_cnt = (torch.argmax(softmax, axis=1) == y_batch).sum()
        Training_accuracy[i] += correct_cnt.item()
    Training_loss[i] /= len(data.dataset)
    #print(Training_accuracy[i])
    Training_accuracy[i] /= len(data.dataset)
    
    print(f"Epoch: {i+1}, Loss: {Training_loss[i]}, Accuracy: {Training_accuracy[i]}")

Epoch: 1, Loss: 0.36034715363581976, Accuracy: 0.8948166666666667
Epoch: 2, Loss: 0.18951323136488596, Accuracy: 0.9445166666666667
Epoch: 3, Loss: 0.15185544942319393, Accuracy: 0.9546166666666667
Epoch: 4, Loss: 0.12887366926968097, Accuracy: 0.96145
Epoch: 5, Loss: 0.11551672877967357, Accuracy: 0.9646166666666667
Epoch: 6, Loss: 0.10378535392483075, Accuracy: 0.9682666666666667
Epoch: 7, Loss: 0.09393494497885306, Accuracy: 0.9707333333333333
Epoch: 8, Loss: 0.08647850543782115, Accuracy: 0.9732333333333333
Epoch: 9, Loss: 0.08094722925697764, Accuracy: 0.9741666666666666
Epoch: 10, Loss: 0.07540133651991686, Accuracy: 0.9759
Epoch: 11, Loss: 0.07160886716991663, Accuracy: 0.9771666666666666
Epoch: 12, Loss: 0.06694994227302571, Accuracy: 0.9784833333333334
Epoch: 13, Loss: 0.06498005988796551, Accuracy: 0.9792666666666666
Epoch: 14, Loss: 0.06001460306271911, Accuracy: 0.9807666666666667
Epoch: 15, Loss: 0.05887327492913852, Accuracy: 0.9814
Epoch: 16, Loss: 0.05450668381763001, A

Possible Error : RuntimeError: result type Float can't be cast to the desired output type Long <br>
Rectified by : correct_cnt.item() in line 15 of above cell

## Test data accuracy

In [11]:
prediction = Model(mnist_test_data.data.to(torch.float32))
prediction.shape

torch.Size([10000, 10])

In [12]:
mnist_test_data.targets.to(torch.float32).shape

torch.Size([10000])

In [13]:
correct_cnt = (
    torch.argmax(prediction, dim=1) == mnist_test_data.targets.to(torch.float32)
    ).sum()

print(f"Test data accuracy: {correct_cnt/len(mnist_test_data.targets) * 100:.2f}%")

Test data accuracy: 96.30%
